# drizzle

In [ ]:
from lovely_docs.settings import settings, GitSource
from lovely_docs.git import clone_repo
from lovely_docs.docs import (
    build_markdown_doc_tree, process_tree_depth_first, calculate_total_usage, calculate_cost,
    save_processed_documents
)
import logging

In [ ]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

#### Clone the repo


In [ ]:
source = GitSource(
    name="drizzle-orm",
    doc_dir="src/content/docs",
    repo="https://github.com/drizzle-team/drizzle-orm-docs",
    commit="main",
    ecosystems=["js"]
)

commit, clone_dir = clone_repo(source)
source.commit = commit  # Replace the literal commit (master) with the hash

#### Read the markdown files


In [ ]:
tree = build_markdown_doc_tree(clone_dir / source.doc_dir, extensions=['.mdx'], exclude=["/latest-releases"])

#### Process the tree


In [ ]:
processed_tree = await process_tree_depth_first(settings, tree, source.name)

+244.170s DEBUG lovely_docs/docs.py:157 llm_process_page Processing column-types/mysql.mdx
+0.261s DEBUG lovely_docs/docs.py:157 llm_process_page Processing column-types/pg.mdx
+0.016s DEBUG lovely_docs/docs.py:157 llm_process_page Processing column-types/singlestore.mdx
+0.015s DEBUG lovely_docs/docs.py:157 llm_process_page Processing column-types/sqlite.mdx
+0.016s DEBUG lovely_docs/docs.py:157 llm_process_page Processing extensions/mysql.mdx
+0.016s DEBUG lovely_docs/docs.py:157 llm_process_page Processing extensions/pg.mdx
+0.020s DEBUG lovely_docs/docs.py:157 llm_process_page Processing extensions/singlestore.mdx
+0.017s DEBUG lovely_docs/docs.py:157 llm_process_page Processing extensions/sqlite.mdx
+0.016s DEBUG lovely_docs/docs.py:157 llm_process_page Processing get-started/bun-sql-existing.mdx
+0.015s DEBUG lovely_docs/docs.py:157 llm_process_page Processing get-started/bun-sql-new.mdx
+0.017s DEBUG lovely_docs/docs.py:157 llm_process_page Processing get-started/bun-sqlite-exis

In [ ]:
usage = calculate_total_usage(processed_tree)
print(f"\nTotal Usage:")
print(f"  Input tokens: {usage.input:,}")
print(f"  Output tokens: {usage.output:,}")
print(f"  Total tokens: {(usage.input + usage.output):,}")

cost, input_cost, output_cost = calculate_cost(usage, 1, 5)

print(f"Cost:")
print(f"  Total: ${cost:.2f}")
print(f"  Input: ${input_cost:.2f}")
print(f"  Output: ${output_cost:.2f}")


Total Usage:
  Input tokens: 880,397
  Output tokens: 260,200
  Total tokens: 1,140,597
Cost:
  Total: $2.18
  Input: $0.88
  Output: $1.30


In [ ]:
save_processed_documents(source, settings.output_dir / source.name.replace("/", "_"), processed_tree)